In [15]:
import random
import pandas as pd
import numpy as np
from sklearn.datasets import make_regression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
from decision_tree import MyTreeReg

class MyForestReg:
    def __init__(self, n_estimators=10, max_features=0.5, max_samples=0.5, random_state=42, max_depth=5, min_samples_split=2, max_leafs=20, bins=16, oob_score=None): 
        # Параметры леса
        self.n_estimators = n_estimators
        self.max_features = max_features
        self.max_samples = max_samples
        self.random_state = random_state
        
        # Параметры деревьев
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_leafs = max_leafs
        self.bins = bins
        
        # Лес (список деревьев)
        self.trees = []
        
        # Словарь для хранения важности признаков
        self.fi = {}  
        
        self.oob_score = oob_score  # параметр oob_score
        self.oob_score_ = None  # Переменная для хранения OOB-ошибки
    
    def __str__(self):
        params = vars(self)  # Получаем все атрибуты экземпляра как словарь
        params_str = ', '.join(f"{key}={value}" for key, value in params.items())
        return f"MyForestReg class: {params_str}"
    
    def fit(self, X: pd.DataFrame, y: pd.Series):
        # Фиксируем сид для воспроизводимости
        random.seed(self.random_state)
        
        # Инициализация параметров
        init_cols = list(X.columns)  # Все колонки
        init_rows_cnt = len(X)  # Количество строк
        
        # Инициализируем важность фичей
        self.fi = {col: 0 for col in init_cols}
        
        # Списки для хранения OOB-выборок
        oob_predictions = np.zeros(init_rows_cnt, dtype=np.float64)
        oob_counts = np.zeros(init_rows_cnt, dtype=np.int32)
        
        # Проходим по количеству деревьев
        for _ in range(self.n_estimators):
            # Выбираем случайные колонки
            cols_smpl_cnt = round(self.max_features * len(init_cols))  # Количество колонок
            cols_idx = random.sample(init_cols, cols_smpl_cnt)  # Индексы колонок
            
            # Выбираем случайные строки
            rows_smpl_cnt = round(self.max_samples * init_rows_cnt)  # Количество строк
            rows_idx = random.sample(range(init_rows_cnt), rows_smpl_cnt)  # Индексы строк
            
            # Формируем подвыборку данных
            X_subset = X.iloc[rows_idx][cols_idx]  # Исправлено
            y_subset = y.iloc[rows_idx]
            
            # Получаем OOB индексы
            oob_mask = np.zeros(init_rows_cnt, dtype=bool)
            oob_mask[rows_idx] = True
            oob_indices = np.where(~oob_mask)[0]
            
            # Создаем дерево с вашим импортированным DecisionTree
            tree = self._create_tree()
            # Обучаем дерево
            tree.fit(X_subset, y_subset, N=init_rows_cnt)
            
            # Предсказываем для OOB-выборки
            if len(oob_indices) > 0:
                oob_preds = tree.predict(X.iloc[oob_indices][cols_idx])
                oob_predictions[oob_indices] += np.array(oob_preds, dtype=np.float64)  # Преобразуем в массив float
                oob_counts[oob_indices] += 1
            
            # Добавляем дерево в лес
            self.trees.append(tree)
            
            # Обновляем важность признаков
            for col in cols_idx:
                self.fi[col] += tree.feature_importances()[col]
                
        # Рассчитываем OOB score
        if self.oob_score is not None:
            valid_mask = oob_counts > 0  # Только те, где были предсказания
            oob_predictions[valid_mask] /= oob_counts[valid_mask]  # Среднее предсказание по OOB
            oob_actuals = y[valid_mask]  # Фактические значения для OOB

            # Используем готовые метрики из sklearn
            if self.oob_score == 'mae':
                self.oob_score_ = mean_absolute_error(oob_actuals, oob_predictions[valid_mask])
            elif self.oob_score == 'mse':
                self.oob_score_ = mean_squared_error(oob_actuals, oob_predictions[valid_mask])
            elif self.oob_score == 'rmse':
                self.oob_score_ = mean_squared_error(oob_actuals, oob_predictions[valid_mask], squared=False)  # RMSE
            elif self.oob_score == 'mape':
                self.oob_score_ = mean_absolute_percentage_error(oob_actuals, oob_predictions[valid_mask]) * 100
            elif self.oob_score == 'r2':
                self.oob_score_ = r2_score(oob_actuals, oob_predictions[valid_mask])
                
    def _create_tree(self):
        # Создаем и возвращаем экземпляр MyTreeReg, используя нужные параметры
        return MyTreeReg(max_depth=self.max_depth, 
                         min_samples_split=self.min_samples_split, 
                         max_leafs=self.max_leafs, 
                         bins=self.bins)

    def predict(self, X):
        # Список для хранения предсказаний от каждого дерева для каждой строки
        predictions = np.zeros(len(X))

        # Проходим по каждому дереву в лесу
        for tree in self.trees:
            tree_preds = tree.predict(X)  # Получаем предсказания от одного дерева
            predictions += np.array(tree_preds)  # Суммируем предсказания
        
        # Возвращаем среднее предсказание
        return predictions / self.n_estimators
    
    def feature_importances(self):
        return self.fi

In [16]:
# Создание данных
X, y = make_regression(n_samples=100, n_features=5, noise=0.1, random_state=42)
X = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(X.shape[1])])
y = pd.Series(y)

# Обучение модели
forest = MyForestReg(n_estimators=3, oob_score='mse')
forest.fit(X, y)

# Вывод OOB score
print(f"OOB MSE: {forest.oob_score_}")


OOB MSE: 19507.103707641694


In [2]:
# важность признаков
X, y = make_regression(n_samples=100, n_features=5, noise=0.1, random_state=42)
X = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(5)])
y = pd.Series(y)

# Создание и обучение случайного леса
forest = MyForestReg(n_estimators=3, max_features=0.8, max_samples=0.7, max_depth=5)
forest.fit(X, y)

# Получение важности всех признаков
print("All feature importances:", forest.feature_importances())

All feature importances: {'feature_0': 565116.5685344362, 'feature_1': 416897.8965800847, 'feature_2': 287938.2001675673, 'feature_3': 68492.72514977847, 'feature_4': 208329.86823288188}


In [29]:
# тест предсказания
X, y = make_regression(n_samples=100, n_features=5, noise=0.1, random_state=42)
X = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(5)])
y = pd.Series(y)

# Создание и обучение случайного леса
forest = MyForestReg(n_estimators=3, max_features=0.8, max_samples=0.7, max_depth=5)
forest.fit(X, y)

# Получение предсказаний
predictions = forest.predict(X)

# Печатаем сумму предсказаний
print("Sum of predictions:", sum(predictions))

Sum of predictions: 1117.1144393196844


In [28]:
X, y = make_regression(n_samples=100, n_features=5, noise=0.1, random_state=42)

# Преобразуем данные в формат DataFrame для совместимости с вашим кодом
X = pd.DataFrame(X, columns=[f'feature{i}' for i in range(X.shape[1])])
y = pd.Series(y)

# Проверка 1: Лес с небольшим количеством деревьев
forest_reg_1 = MyForestReg(n_estimators=5, random_state=42)
forest_reg_1.fit(X, y)
print(f"Проверка 1 - Количество листьев: {forest_reg_1.leafs_cnt}")

# Проверка 2: Лес с большим количеством деревьев
forest_reg_2 = MyForestReg(n_estimators=20, max_depth=7, random_state=42)
forest_reg_2.fit(X, y)
print(f"Проверка 2 - Количество листьев: {forest_reg_2.leafs_cnt}")

# Проверка 3: Лес с измененным количеством фич и сэмплов для каждого дерева
forest_reg_3 = MyForestReg(n_estimators=10, max_features=0.7, max_samples=0.8, random_state=42)
forest_reg_3.fit(X, y)
print(f"Проверка 3 - Количество листьев: {forest_reg_3.leafs_cnt}")



Проверка 1 - Количество листьев: 79
Проверка 2 - Количество листьев: 403
Проверка 3 - Количество листьев: 204


In [2]:
# Тестирование класса
forest1 = MyForestReg()
forest2 = MyForestReg(n_estimators=10, max_features=0.5, max_samples=0.5, random_state=42, max_depth=5, min_samples_split=5, max_leafs=20, bins=16)
forest3 = MyForestReg(n_estimators=8, max_features=0.5, max_samples=0.5, random_state=42, max_depth=5, min_samples_split=5, max_leafs=20, bins=16)

# Проверка
print(forest1)
print(forest2)
print(forest3)

MyForestReg class: n_estimators=10, max_features=0.5, max_samples=0.5, random_state=42, max_depth=5, min_samples_split=2, max_leafs=20, bins=16, trees=[], leafs_cnt=0
MyForestReg class: n_estimators=10, max_features=0.5, max_samples=0.5, random_state=42, max_depth=5, min_samples_split=5, max_leafs=20, bins=16, trees=[], leafs_cnt=0
MyForestReg class: n_estimators=8, max_features=0.5, max_samples=0.5, random_state=42, max_depth=5, min_samples_split=5, max_leafs=20, bins=16, trees=[], leafs_cnt=0
